In [1]:
%load_ext autoreload
%autoreload 2

In [211]:
from wdtools import *
import fiona
import webbrowser

In [3]:
revpath = inpath + '\GIS\ArcGIS Pro Project\DataReview\DataReview.gdb'

In [11]:
setID = 'Set003'

In [5]:
nm_to_add = 2143

In [12]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [42]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [43]:
len(wd.wetdet_delin_number.unique())

433

In [14]:
matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')

In [137]:
mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]

In [16]:
partial = gpd.read_file(revpath, layer=f'{setID}_partial')

In [17]:
def review_mapped(setID):
    mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]
    for wID in mapped0:
        gdf = gpd.read_file(revpath, layer=wID)
        if 'wdID' in gdf.columns:
            if len(gdf.wdID.unique()) > 1:
                print(wID)

In [18]:
def revise_single_partial_file(wID):
    gdf = gpd.read_file(revpath, layer=wID)
    selcols = ['Shape_Length', 'Shape_Area']
    if 'wdID' in gdf.columns:
        df = gdf.copy()[selcols +  ['wdID']]
    else:
        df = gdf.copy()[selcols]
        df.loc[:,'wdID'] = wID.replace('_', '-')
    df.loc[:,'geometry'] = gdf.loc[:,'geometry']
    return df

In [19]:
def merge_single_partial_file(wIDlist):
    df = pd.DataFrame()
    for wID in wIDlist:
        df=pd.concat([df, revise_single_partial_file(wID)], ignore_index=True)
    gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')
    return gdf

In [201]:
def combine_matched_digitized(setID, editedIDs, nm_to_add):
    mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]
    matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')
    partial = gpd.read_file(revpath, layer=f'{setID}_partial')
    gdf = merge_single_partial_file(mapped0)
    dat = gdf.append(partial, ignore_index=True)
    edited_gdf = matched[matched.wdID.isin(editedIDs)]
    edited_gdf = edited_gdf[['wdID', 'geometry']].dissolve('wdID')
    edited_gdf.loc[:, 'wdID'] = edited_gdf.index
    data1 = edited_gdf.append(dat[['wdID', 'geometry']], ignore_index=True)
    matched_df = matched[['wdID', 'geometry']].dissolve('wdID')
    matched_df.loc[:, 'wdID'] = matched_df.index
    excluded = [wdID for wdID in data1.wdID.unique() if wdID in matched_df.wdID.unique()]
    issues = pd.read_csv(os.path.join(inpath, "output", "to_review", f"{setID}_Mapping_Issues.csv"))
    issueIDs = list(issues.wetdet_delin_number.unique())
    matched_gdf = matched_df[~matched_df.wdID.isin(excluded+issueIDs)]
    data2 = matched_gdf.append(data1, ignore_index=True)
    wo_lot = gpd.read_file(revpath, layer=f'{setID}_wo_lot')
    wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)
    df = wd[wd.record_ID.isin(wo_lot.record_ID)][['wetdet_delin_number', 'record_ID']]
    df.rename(columns={'wetdet_delin_number': 'wdID'}, inplace=True)
    wo_lot = wo_lot.merge(df, on='record_ID')
    data3 = data2.append(wo_lot[['wdID', 'geometry']], ignore_index=True)
    return data3, issueIDs

In [20]:
revise_single_partial_file('WD2019_0130')

,Shape_Length,Shape_Area,wdID,geometry
0,14604.540094,8.568226e+06,WD2019-0130,"MULTIPOLYGON (((710392.945 1475437.580, 710804..."


In [173]:
edited = ['WD2018-0261', 'WD2018-0422', 'WD2018-0635', 'WD2018-0642', 'WD2019-0048', 
          'WD2019-0369', 'WD2020-0507', 'WD2021-0536', 'WD2021-0590', 'WD2021-0700']

In [202]:
res = combine_matched_digitized(setID=setID, editedIDs=edited, nm_to_add=nm_to_add)

In [203]:
gdf = res[0]

In [192]:
len(gdf.wdID.unique())

422

In [199]:
unmatchedIDs = [wdID for wdID in wd.wetdet_delin_number.unique() if wdID not in gdf.wdID.unique()]

In [204]:
issueIDs = res[1]

In [205]:
[ID for ID in unmatchedIDs if ID not in issueIDs]

['WD2018-0485', 'WD2019-0060', 'WD2021-0181']

In [ ]:
wdID = 'WD2021-0181'
url = wd.loc[wd.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
webbrowser.open(url)

In [216]:
url

nan

In [217]:
wd.loc[wd.wetdet_delin_number == wdID, 'record_ID']

659    2803
Name: record_ID, dtype: int32

In [ ]:
extract_page_from_docLink(url=url, pageNm=4, wdID=wdID)

In [214]:
extract_page_from_locPath(filePath=r'C:\Users\clid1852\Downloads\WD20190060 AgencyDecision.pdf', #T:\DCProjects\EPA-WD\docs\
                          pageNm=8, wdID=wdID)

In [47]:
tomapID = matched[matched.notes.str.contains('Partial', case=False, na=False)].wdID.unique()